![Dockerlogo](Pictures/podman-logo.png)  

# What is Podman?

As described by the Podman website itself it is:

    Podman is a daemonless, open source, Linux native tool designed to make it easy to find, run, build, share and deploy applications using Open Containers Initiative (OCI) Containers and Container Images. Podman provides a command line interface (CLI) familiar to anyone who has used the Docker Container Engine. Most users can simply alias Docker to Podman (alias docker=podman) without any problems. Similar to other common Container Engines (Docker, CRI-O, containerd), Podman relies on an OCI compliant Container Runtime (runc, crun, runv, etc) to interface with the operating system and create the running containers. This makes the running containers created by Podman nearly indistinguishable from those created by any other common container engine.

With simple words it's a container engine that is so similar to Docker that you can just use an alias and run docker-like commands. This brings to mind an obvious question, if it's so similar to Docker why would you use Podman instead? The answer is security.
The previous paragraph starts describing Podman as a "daemonless" container engine and that's the main difference with Docker. Later in this workshop we'll explain security features in Podman, for the moment just keep in mind that as Podman doesn't have a daemon it doesn't need to use the root user for anything, therefore the execution of containers is secured by design.

This is explained differently in Podman website:

    Containers under the control of Podman can either be run by root or by a non-privileged user. Podman manages the entire container ecosystem which includes pods, containers, container images, and container volumes using the libpod library. Podman specializes in all of the commands and functions that help you to maintain and modify OCI container images, such as pulling and tagging. It allows you to create, run, and maintain those containers and container images in a production environment.

# Why Podman?

Many think of Podman to be a replacement for Docker (if they have heard of Podman at all). But, this is not the case, as Podman is another option that provides better security and developer features. Podman is a cloud-native, daemonless tool that helps developers manage their Linux containers. Podman is all about security, but also minimizing the friction between your local development environment and production.

Podman uses a microservices approach, creating a network with many other cloud-native products, such as Buildah and Skopeo, to build and push containers. This makes Podman a lighter and faster application than Docker, allowing for customization and changes.

# Podman basic functionality: managing container images

Just like any other container engine, Podman has the ability to manage and execute container images. A container image is a static image that contains an applications with all it's dependencies. Because the dependencies are included within the container image we can expect the same application behaviour if it's executed in different hosts, solving one of the most widespread problems for developers: "it worked in my environment".

Container images are stored in container registries, by default Podman will look for container images in quay.io, registry.redhat.io and docker.io. You can configure additional registries by modifying the file /etc/containers/registries.conf, we'll not cover this advanced configuration during this workshop.

The way Podman works with container images is it locally downloads (or pulls) images from registry. Hence, first thing we want to do is checking if we have any image downloaded locally:

In [10]:
!podman images

REPOSITORY                                                          TAG         IMAGE ID      CREATED        SIZE
docker.io/library/nginx                                             latest      61395b4c586d  2 weeks ago    191 MB
localhost/bcl-ov                                                    15          ef3ba4fe4c30  4 weeks ago    717 MB
<none>                                                              <none>      2be670a5b439  4 weeks ago    803 MB
<none>                                                              <none>      7a4c1f405114  4 weeks ago    369 MB
quay.io/podman/hello                                                latest      e2b3db5d4fdf  5 weeks ago    82.3 kB
quay.io/mschreie/bcl-ov                                             14          a3e1ce7d79a7  7 weeks ago    717 MB
<none>                                                              <none>      af6827d3105f  7 weeks ago    299 MB
quay.io/redhat_emp1/ee-ansible-ssa                                  lates

You should get have no images locally. Download your first image, the podman hello-world image, by running the following command:

In [11]:
!podman pull hello-world

Trying to pull quay.io/podman/hello:latest...
Getting image source signatures
Copying blob d08b40be6878 skipped: already exists  
Copying blob d08b40be6878 skipped: already exists  
Copying config e2b3db5d4f done   | 
Copying config e2b3db5d4f done   | 
Copying config e2b3db5d4f done   | 
Writing manifest to image destination
e2b3db5d4fdf670b56dd7138d53b5974f2893a965f7d37486fbb9fcbf5e91d9d


Now check if your images was pulled correctly by running again the podman images command:

In [12]:
!podman images

REPOSITORY                                                          TAG         IMAGE ID      CREATED        SIZE
docker.io/library/nginx                                             latest      61395b4c586d  2 weeks ago    191 MB
localhost/bcl-ov                                                    15          ef3ba4fe4c30  4 weeks ago    717 MB
<none>                                                              <none>      2be670a5b439  4 weeks ago    803 MB
<none>                                                              <none>      7a4c1f405114  4 weeks ago    369 MB
quay.io/podman/hello                                                latest      e2b3db5d4fdf  5 weeks ago    82.3 kB
quay.io/mschreie/bcl-ov                                             14          a3e1ce7d79a7  7 weeks ago    717 MB
<none>                                                              <none>      af6827d3105f  7 weeks ago    299 MB
quay.io/redhat_emp1/ee-ansible-ssa                                  lates

You should find the hello-world image is locally downloaded in your system. This only means that we have a static image that contains our application and its dependancies locally sync'ed. Next thing we would like to do is to execute that static image:

In [13]:
!podman run hello-world

!... Hello Podman World ...!

         .--"--.           
       / -     - \         
      / (O)   (O) \        
   ~~~| -=(,Y,)=- |         
    .---. /`  \   |~~      
 ~/  o  o \~~~~.----. ~~   
  | =(X)= |~  / (O (O) \   
   ~~~~~~~  ~| =(Y_)=-  |   
  ~~~~    ~~~|   U      |~~ 

Project:   https://github.com/containers/podman
Website:   https://podman.io
Documents: https://docs.podman.io
Twitter:   @Podman_io


# Executing containerized applications

During last exercise we downloaded a container image and executed it. With the following command Podman will show the containers that are being executed right now:

In [14]:
!podman ps

CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES


As you can see, the output is empty. This is because the container that we executed runs a process that ends after printing in the screen the "Hello Podman World" message and once the main process of a container is finished the container itself will stop being executed.
But no worries, Podman got us covered, if we want to see all the containers that are being executed and those whose execution finished some time ago we just need to add the "-a" to the previous command.

In [15]:
!podman ps -a

CONTAINER ID  IMAGE                        COMMAND               CREATED         STATUS                     PORTS       NAMES
5fd82af4a43c  quay.io/podman/hello:latest  /usr/local/bin/po...  11 seconds ago  Exited (0) 11 seconds ago              admiring_benz


There you can see the container we executed before with a status of "Exited".

Lets see what happens if we execute a container whose process don't finish unless specified:

In [16]:
!podman run nginx

/docker-entrypoint.sh: /docker-entrypoint.d/ is not empty, will attempt to perform configuration
/docker-entrypoint.sh: Looking for shell scripts in /docker-entrypoint.d/
/docker-entrypoint.sh: Launching /docker-entrypoint.d/10-listen-on-ipv6-by-default.sh
10-listen-on-ipv6-by-default.sh: info: Getting the checksum of /etc/nginx/conf.d/default.conf
10-listen-on-ipv6-by-default.sh: info: Enabled listen on IPv6 in /etc/nginx/conf.d/default.conf
/docker-entrypoint.sh: Sourcing /docker-entrypoint.d/15-local-resolvers.envsh
/docker-entrypoint.sh: Launching /docker-entrypoint.d/20-envsubst-on-templates.sh
/docker-entrypoint.sh: Launching /docker-entrypoint.d/30-tune-worker-processes.sh
/docker-entrypoint.sh: Configuration complete; ready for start up
2023/10/06 08:09:00 [notice] 1#1: using the "epoll" event method
2023/10/06 08:09:00 [notice] 1#1: nginx/1.25.2
2023/10/06 08:09:00 [notice] 1#1: built by gcc 12.2.0 (Debian 12.2.0-14) 
2023/10/06 08:09:00 [notice] 1#1: OS: Linux 6.5.5-200.fc38.

You need to stop the process manually as the container process has taken all of your terminal. If you want to run a container in the background, then you should add the option --detach (or simply -d).

In [20]:
!podman run --detach nginx

329d69e838275ed59a09a8d639920bc3282205fb597d98b03201723fcebc6d26


Now, if you run the "podman ps" command you will see your nginx container with a status of "Up".

In [21]:
!podman ps

CONTAINER ID  IMAGE                           COMMAND               CREATED         STATUS         PORTS       NAMES
329d69e83827  docker.io/library/nginx:latest  nginx -g daemon o...  33 seconds ago  Up 33 seconds              determined_gates


You see the container has an ID and also a name. In this case the name was randomly generated as we did not specify any name, pay attention to next time we execute a container image as we will be giving it a name.

You can stop all running containers by running the following:

In [23]:
!podman stop --all

357634993c693ebef18240a4b6a1a0e9f3c38f5a5385cf92ec7c77503bb0b18a
5fd82af4a43cec98c871f7c606cfd27a8ade0c173ae54f1f5e1d48cd12e30338
329d69e838275ed59a09a8d639920bc3282205fb597d98b03201723fcebc6d26


Now that all containers have beeen stopped run again "podman ps" to check if all of them were correctly stopped.

In [27]:
!podman ps

CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES


As you can see the container is not present because "podman ps" only shows the containers that are being executed right now. If we use the "-a" option it will show all containers that are in execution right now but also those that were stopped, it'll show the stopped ones with an status of "Exited".

In [26]:
!podman ps -a

CONTAINER ID  IMAGE                           COMMAND               CREATED            STATUS                         PORTS       NAMES
5fd82af4a43c  quay.io/podman/hello:latest     /usr/local/bin/po...  About an hour ago  Exited (0) About an hour ago               admiring_benz
357634993c69  docker.io/library/nginx:latest  nginx -g daemon o...  About an hour ago  Exited (0) 44 minutes ago                  bold_lichterman
329d69e83827  docker.io/library/nginx:latest  nginx -g daemon o...  41 minutes ago     Exited (0) About a minute ago              determined_gates


This means that containers are not deleted once the process is finished and you could simply restart any of them. We can do this by using the container ID, but for the shake of simplicity we will be creating a new container with a defined name:

In [40]:
!podman run -d --name my-podman-container nginx
!podman ps

2cbb3e8ff09a77b48ab0278144b6c52f7a23c08120e79629b3d180db36101271
CONTAINER ID  IMAGE                           COMMAND               CREATED                 STATUS                 PORTS       NAMES
2cbb3e8ff09a  docker.io/library/nginx:latest  nginx -g daemon o...  Less than a second ago  Up Less than a second              my-podman-container


See how we changed the long "--detach" option for a smaller "-d" but it works the same way. We also defined the name. So now we can just stop it by it's name.

In [34]:
!podman stop my-podman-container
!podman ps -a

my-podman-container
CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES
CONTAINER ID  IMAGE                           COMMAND               CREATED             STATUS                     PORTS       NAMES
d9372a2648eb  docker.io/library/nginx:latest  nginx -g daemon o...  About a minute ago  Exited (0) 15 seconds ago              my-podman-container


Now that it is stopped, we can restart it.

In [39]:
!podman start my-podman-container
!podman ps

Error: no container with name or ID "my-podman-container" found: no such container


CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES


While the container is running you can also inspect it to get to know all the information about it.

In [42]:
!podman inspect my-podman-container

[
     {
          "Id": "2cbb3e8ff09a77b48ab0278144b6c52f7a23c08120e79629b3d180db36101271",
          "Created": "2023-10-06T12:04:57.156652259+02:00",
          "Path": "/docker-entrypoint.sh",
          "Args": [
               "nginx",
               "-g",
               "daemon off;"
          ],
          "State": {
               "OciVersion": "1.1.0+dev",
               "Status": "running",
               "Running": true,
               "Paused": false,
               "Restarting": false,
               "OOMKilled": false,
               "Dead": false,
               "Pid": 45203,
               "ConmonPid": 45201,
               "ExitCode": 0,
               "Error": "",
               "StartedAt": "2023-10-06T12:04:57.291749039+02:00",
               "FinishedAt": "0001-01-01T00:00:00Z",
               "Health": {
                    "Status": "",
                    "FailingStreak": 0,
                    "Log": null
               },
               "CgroupPath": "/user.slic

If we stop it again we see that we can also remove it by running the command "podman rm"

In [37]:
!podman stop my-podman-container
!podman rm my-podman-container

my-podman-container
my-podman-container


Now it has been fully removed so we wont be able to start that same container again, we will need to create a new container from the same image. You can check it is fully removed by looking at the empty output of "podman ps -a"

In [38]:
!podman ps -a

CONTAINER ID  IMAGE       COMMAND     CREATED     STATUS      PORTS       NAMES
